In [1]:
from platform import python_version
print("Python version: ", python_version())

import numpy as np
print("Numpy version: ", np.__version__)

import os
current_path = os.getcwd()
PETITE_home_dir= current_path.split('examples')[0]

print("PETITE home directory:", PETITE_home_dir)

dictionary_dir = "data/"

Python version:  3.8.5
Numpy version:  1.24.4
PETITE home directory: /Users/taozhou/Documents/GitHub/PETITE/


In [2]:
# import src
import sys
sys.path.append(PETITE_home_dir)
from src.dark_shower import *
from src.shower import *
import pickle as pk
from matplotlib import pyplot as plt
from tqdm import tqdm
#from tqdm.notebook import tqdm

import matplotlib

from matplotlib.font_manager import FontProperties
from matplotlib.ticker import FixedLocator, MaxNLocator
import cProfile
profile = cProfile.Profile()
import pstats

font0 = FontProperties()
font = font0.copy()
font.set_size(24)
font.set_family('serif')
labelfont=font0.copy()
labelfont.set_size(20)
labelfont.set_weight('bold')
legfont=font0.copy()
legfont.set_size(18)
legfont.set_weight('bold')


def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)

In [6]:
# Set up lists to organize data according to BSM process and parent particle
Energies, Weights, particles, totalflux = {}, {}, {}, {}
parent_pid_options = [11, -11, 22, 111]
for pr in dark_process_codes:
    Energies[pr] = {}
    Weights[pr] = {}
    particles[pr] = {}
    totalflux[pr] = {}
    for pid in parent_pid_options:
        Energies[pr][pid] = []
        Weights[pr][pid] = []
        particles[pr][pid] = []
        totalflux[pr][pid] = []

In [5]:
data = np.loadtxt(PETITE_home_dir + dictionary_dir + 'Pi0_400GeVProtons_1e3POT.txt')
E = data[:,0]
px = data[:,1]
py = data[:,2]
pz = data[:,3]
POT = 1e3
mass_list = np.logspace(-3, 0, 100)
shower_list = []
for i in tqdm(range(len(mass_list))):
    sTungsten = DarkShower(PETITE_home_dir+dictionary_dir, 0.030, mass_list[i], kinetic_mixing=1.0)
    for i in tqdm(range(len(data))):
        p0 = [E[i], px[i], py[i], pz[i]]
        r0 = [0, 0, 0]
        pdict = {"PID":111, "weight":1.0/POT, "stability":"short-lived"}
        part0 = Particle(p0, r0, pdict)
        s0SM = sTungsten.generate_shower(part0)
        s0BSM = sTungsten.generate_dark_shower(ExDir=list(s0SM))
        for p0 in s0BSM[1]:   
            genprocess = p0.get_ids()["generation_process"]
            parent_pid = p0.get_ids()["parent_PID"]

            Energies[genprocess][parent_pid].append(p0.get_p0()[0])
            Weights[genprocess][parent_pid].append(p0.get_ids()["weight"])
            particles[genprocess][parent_pid].append(p0)

#stungsten = DarkShower(PETITE_home_dir+dictionary_dir, "tungsten", 0.030, 0.01, kinetic_mixing=1.0)

[0.001      0.00107227 0.00114976 0.00123285 0.00132194 0.00141747
 0.00151991 0.00162975 0.00174753 0.00187382 0.00200923 0.00215443
 0.00231013 0.00247708 0.00265609 0.00284804 0.00305386 0.00327455
 0.00351119 0.00376494 0.00403702 0.00432876 0.00464159 0.00497702
 0.0053367  0.00572237 0.00613591 0.00657933 0.0070548  0.00756463
 0.00811131 0.00869749 0.00932603 0.01       0.01072267 0.01149757
 0.01232847 0.01321941 0.01417474 0.01519911 0.01629751 0.01747528
 0.01873817 0.02009233 0.02154435 0.0231013  0.02477076 0.02656088
 0.02848036 0.03053856 0.03274549 0.03511192 0.03764936 0.04037017
 0.04328761 0.04641589 0.04977024 0.05336699 0.05722368 0.06135907
 0.06579332 0.07054802 0.07564633 0.08111308 0.0869749  0.09326033
 0.1        0.10722672 0.1149757  0.12328467 0.13219411 0.14174742
 0.15199111 0.16297508 0.17475284 0.18738174 0.2009233  0.21544347
 0.23101297 0.24770764 0.26560878 0.28480359 0.30538555 0.32745492
 0.35111917 0.37649358 0.40370173 0.43287613 0.46415888 0.4977